In [2]:
from gensim import corpora, models, similarities
import pickle
import string
import pandas as pd

In [3]:
def remove_punctuation(x):
    x = str(x)
    return x.translate(string.maketrans('',''),string.punctuation)

In [4]:
beers = pd.read_pickle('all_beer_reviews.pkl')

In [6]:
documents = [review for review in beers.reviews]

In [8]:
documents = [' '.join(review) for review in documents]

In [9]:
documents = [remove_punctuation(doc) for doc in documents]

In [10]:
documents = [review.lower() for review in documents]

In [20]:
stoplist = set('an one little just has be up had no with is this it i but that on not very some as was like from its bit at more into there my pours for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]for document in documents]

In [21]:
from collections import defaultdict

In [104]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 5]for text in texts]

[('gmcthis', 1),
 ('inchvery', 1),
 ('rescuenice', 1),
 ('bmanningpoured', 1),
 ('wineive', 1),
 ('juiceskin', 1),
 ('lagerdefinitely', 1),
 ('lighttight', 1),
 ('whilecoffee', 1),
 ('inlawinlaw', 1),
 ('alsocarbonation', 1),
 ('peachtons', 1),
 ('bitternessreceptive', 1),
 ('flavors\xc2\xa0i', 1),
 ('5987', 1),
 ('beijingdated', 1),
 ('hopfiend', 1),
 ('vollabel', 1),
 ('nellow', 1),
 ('coffeetasterich', 1),
 ('neutralhideous', 1),
 ('camparim', 1),
 ('90bottle', 1),
 ('woodl', 1),
 ('hangingoutwithfriends', 1),
 ('2007olde', 1),
 ('28714a', 1),
 ('withasting', 1),
 ('zeltbier', 1),
 ('burnt35', 1),
 ('weell', 1),
 ('cervoisesplitting', 1),
 ('drinkinginitial', 1),
 ('32416look', 1),
 ('promsied', 1),
 ('fourstar', 1),
 ('prevelentwhat', 1),
 ('spiceyhoppish', 1),
 ('vnious', 1),
 ('sonspours', 1),
 ('awhilewell', 1),
 ('woodey', 1),
 ('cloudymetallic', 1),
 ('summerdrinker', 1),
 ('080612', 1),
 ('bbqy', 1),
 ('welloverallenjoyable', 1),
 ('powdertasteherbal', 1),
 ('goldencreamy', 1

In [105]:
dictionary = corpora.Dictionary(texts)

In [106]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [107]:
tfidf = models.TfidfModel(corpus)

In [108]:
corpus_tfidf = tfidf[corpus]

In [109]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=500)

In [110]:
index = similarities.MatrixSimilarity(lsi[corpus])

In [51]:
beers[beers.name == 'Heady Topper'].index[0]

1084

In [111]:
beers = beers.reset_index()

In [127]:
text_input = 'Death & Taxes Black Beer'

In [128]:
# get the reviews for a beer
beer_name_inputted = 1
try:
    doc= documents[beers[beers.name == text_input].index[0]]
except IndexError:
    print 'Beer Name Not Inputted'
    doc = text_input
    beer_name_inputted = 0
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]

sims = index[vec_lsi]
similar_beers = []
for beer in sorted(enumerate(sims), key = lambda x: -x[1])[beer_name_inputted:beer_name_inputted+5]:
    similar_beers.append(beers.name[beer[0]])
    #print(beers.name.iloc[beer[0]] + '\t:\t%.2f' % (beer[1]*100))

Beer Name Not Inputted


In [36]:
# documents
# dictionary
# lsi    41.2s
# index  6.4s --rebuild on app

In [120]:
pickle.dump(documents,open('flask/app/models/documents.pkl','wb'))

In [121]:
pickle.dump(dictionary,open('flask/app/models/dictionary.pkl','wb'))

In [122]:
pickle.dump(lsi,open('flask/app/models/lsi.pkl','wb'))

In [123]:
pickle.dump(corpus,open('flask/app/models/corpus.pkl','wb'))

In [124]:
pickle.dump(index,open('flask/app/models/index.pkl','wb'))

In [125]:
beers.to_pickle('flask/app/models/beer_review_df.pkl')

In [55]:
pickle.dump(beer_names,open('flask/app/models/beer_names.pkl','wb'))

In [1]:
from sklearn.cluster import KMeans